In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import base64
from datetime import datetime

In [ ]:
data = pd.read_csv("data/parsed_data2.csv")

In [ ]:
X = np.array([np.frombuffer(base64.b64decode(board), dtype=np.bool_).reshape((6,7,2)) for board in data["board"]])
y = data["evaluation"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=(6, 7, 2)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input((6,7,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.Conv2D(256, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(1, activation="linear")
])
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

In [ ]:
model = tf.keras.models.load_model("models/model2/2024-04-04_11-09-58.keras")
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=2, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

In [ ]:
for test_board in ["AAAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAQAAAAAAAAABAAABAAEAAAAAAAAA",  #-1
                   "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAEAAAAAAAAA",  #-1
                   "AAAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAQAAAAAAAAABAAEBAAEAAAAAAAAA",  #1
                   "AAAAAAAAAAEAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAEAAQEAAAABAAAAAAABAAEAAQAAAAABAAABAAABAQAAAQAAAAEAAAEAAAEBAAEAAAEAAQAA"]: #-10
    fbtest_board = np.frombuffer(base64.b64decode(test_board), dtype=np.bool_).reshape((6,7,2))
    edtest_board = np.expand_dims(fbtest_board, axis=0)
    print(model.predict(edtest_board))

In [ ]:
model.save(f"models/fromdata2/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")